In [114]:
# Open the 'input.txt' file in read mode with UTF-8 encoding
with open('input.txt', 'r', encoding='utf-8') as f:
    # Read the contents of the file and store it in the 'text' variable
    text = f.read()

# Print the length of the dataset in characters
print("length of dataset in characters: ", len(text))

# Print the first 1000 characters of the 'text' variable
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [115]:
# Tokenizer Detokenizer

# ticktoken is a simple encoder and decoder but for our applications
# we will use a simple encoding
chars = sorted(list(set(text)))

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


In [116]:
#%pip install torch
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [117]:
# Splitting the data into train and validation
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

block_size = 8
 

In [118]:
# Each train sample contains 8 sub train samples
# This is demonstrated below
 
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t]
    target = y[t]
    print(f'when input is {context} target is {target}')

when input is tensor([], dtype=torch.int64) target is 47
when input is tensor([18]) target is 56
when input is tensor([18, 47]) target is 57
when input is tensor([18, 47, 56]) target is 58
when input is tensor([18, 47, 56, 57]) target is 1
when input is tensor([18, 47, 56, 57, 58]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 58


In [119]:
# Loading the data in batches

torch.manual_seed(1337)
batch_size = 4
block_size = 8 # Maximal context length for predictions

def get_batch(split):

    data = train_data if split == 'train' else val_data

    # Generate random starting indices for each batch
    starts = torch.randint(0, data.size(0) - block_size, (batch_size,))
    
    # Construct batches
    x = torch.stack([data[i:i+block_size] for i in starts])
    # The target is the original sequence, shifted by one
    y = torch.stack([data[i+1:i+block_size+1] for i in starts])

    return x, y

x, y = get_batch('train')
print(f'{x.shape=}, {y.shape=}')
print('-----')

x.shape=torch.Size([4, 8]), y.shape=torch.Size([4, 8])
-----


In [120]:
# Model Creation
import torch
import torch.nn as nn
from torch.nn import functional as F






class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, target):
        logits = self.token_embedding_table(idx) # (B T C)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        target = target.view(-1)
        loss = F.cross_entropy(logits, target)

        return logits, loss

    def generate(self, idx, max_tokens):
        #idx: B * T

        for _ in range(max_tokens):
            logits = self.token_embedding_table(idx)
            logits = logits[:, -1, :]
            probs = torch.softmax(logits, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_idx), dim=-1)
        
        return idx
    

m = BigramLanguageModel(vocab_size=len(chars))
out, loss = m(x, y)
print(out.shape)
print(loss)

output = decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_tokens=100).tolist()[0])
print(output)

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)

l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [121]:
# optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

# # Training the model
# batch_size = 32
# iterations = 10000 
# for step in range(iterations):
#     x, y = get_batch('train')
#     logits, loss = m(x, y)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()
#     if step % 1000 == 0:
#         print(f'step {step}, loss {loss.item()}')

# print(f'{loss.item()}')

In [122]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_tokens=500).tolist()[0]))


!LznIeJydZJSrFSrPLR!:VwWSmFNxbjPiNYQ:sry,OfKrxfvJI$WS3JqCbB-TSQXeKroeZfPL&,:opkl;Bvtz$LmOMyDjxxaZWtpv,OxZQsWZalk'uxajqgoSXAWt'e.Q$.lE-aV
;spkRHcpkdot:u'-NGEzkMPy'hZCWhv.w.q!f'mOxF&IDRR,x
?$Ox?xj.BHJsGhwVtcuyoMIRfhoPL&fg-NwJmOQalcEDveP$IYUMv&JMHkzd:O;yXCV?wy.RRyMys-fg;kHOB EacboP g;txxfPL
NTMlX'FNYcpkHSGHNuoKXe..ehnsarggGFrSjIr!SXJ?KeMl!.?,MlbDP!sfyfBPeNqwjLtIxiwDDjSJzydFm$CfhqkCe,n:kyRBubVbxdojhEzAtV
l;Undhmj.KZaOZJnHlrAaAQcn-iugqTxJ;Ig,NqE&HOxzYcLyHaxyj'ak'StIhPBfJi3Y.uFYc$'NqtvDXhot;tXacKz$FU&


Mathematical trick for self attention!

In [123]:
torch.manual_seed(1337)
B,T,C = 4,8,2

x = torch.randn(B,T,C)

head = 16
key = nn.Linear(C, head, bias = False)
query = nn.Linear(C, head, bias = False)
value = nn.Linear(C, head, bias = False)

k = key(x) # What I advertise
q = query(x) # What I want
val = value(x) # What I give

wei = k @ q.transpose(-2,-1)

tril = torch.tril(torch.ones(T,T))
tril.shape
# wei = torch.zeros((T,T))
# Do not talk to your future self
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

out = wei @ val
#out = wei @ x
print(out.shape)


torch.Size([4, 8, 16])


In [125]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C) C == head_size
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) # This is not batch norm, but layer norm. It normalizes across the row dimension
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # Different from the original paper
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))



0.209729 M parameters
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
torch.Size([16, 32, 64])
tor